# Project 2: Web Traffic Analysis
**This is the second of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-11-10, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
In this project your task is to analyze a stream of log entries. A log entry consists of an [IP address](https://en.wikipedia.org/wiki/IP_address) and a [domain name](https://en.wikipedia.org/wiki/Domain_name). For example, a log line may look as follows:

`192.168.0.1 somedomain.dk`

One log line is the result of the event that the domain name was visited by someone having the corresponding IP address. Your task is to analyze the traffic on a number of domains. Counting the number of unique IPs seen on a domain doesn't correspond to the exact number of unique visitors, but it is a good estimate.

Specifically, you should answer the following questions from the stream of log entries.

- How many unique IPs are there in the stream?
- How many unique IPs are there for each domain?
- How many times was IP X seen on domain Y? (for some X and Y provided at run time)

**The answers to these questions can be approximate!**

You should also try to answer one or more of the following, more advanced, questions. The answers to these should also be approximate.

- How many unique IPs are there for the domains $d_1, d_2, \ldots$?
- How many times was IP X seen on domains $d_1, d_2, \ldots$?
- What are the X most frequent IPs in the stream?

You should use algorithms and data structures that you've learned about in the lectures, and you should provide your own implementations of these.

Furthermore, you are expected to:

- Document the accuracy of your answers when using algorithms that give approximate answers
- Argue why you are using certain parameters for your data structures

This notebook is in three parts. In the first part you are given an example of how to read from the stream (which for the purpose of this project is a remote file). In the second part you should implement the algorithms and data structures that you intend to use, and in the last part you should use these for analyzing the stream.

## Reading the stream
The following code reads a remote file line by line. It is wrapped in a generator to make it easier to extend. You may modify this if you want to, but your solution should remain parametrized, so that your notebook can be run without having to consume the entire file.

In [1]:
import urllib

def stream(n):
    i = 0
    with urllib.request.urlopen('https://files.dtu.dk/fss/public/link/public/stream/read/traffic_2?linkToken=_DcyO-U3MjjuNzI-&itemName=traffic_2') as f:
        for line in f:
            element = line.rstrip().decode("utf-8")
            yield element
            i += 1
            if i == n:
                break

In [2]:
STREAM_SIZE = 1000
web_traffic_stream = stream(STREAM_SIZE)

#for e in web_traffic_stream:
    #print(e)

## Data structures

In [3]:
import mmh3
import random
import sys
import math
import statistics as s

m = 64
# hashes element with a seed from an arbritary large input to a 64 bit output
def hashElement(element, seed):
    # mmh3 64 outputs a 128 bit hash split in to two 64 bit outputs. 
    #For this purpose returning one of them will be sufficient
    a,b = mmh3.hash64(element, seed, signed=False)
    return a


def addElement(element, M):
    hashE = hashElement(eToHash, seed)
    upperpart = hashE >> size_lower_part
    lowerpart = hashE & ((2 ** size_lower_part) -1)
    # p is the position of the left most bit
    # We subtract it for 64 to get the position counting from left to right
    p = 64 - int(math.log(lowerpart,2))
    # j is the integer representation of the upperpart, we do not add one as python has 0 indexed 
    j = upperpart 
    # Use upper part as index in M and set it to the max of M[j] and p
    M[j] = max(M[j],p)
    return M

# Method to build the HLL data structure
# If true for rnd, take a random seed
# if true for domain, hash the domain part of the element
def buildHyperLogLog(stream, streamSize, rnd, domain, m):
    # define size of M
    M = [0]*m
    # If rnd is false, use default seed
    seed = 42
    if(rnd):
        seed = random.randint(0,sys.maxsize/2)
    # Defines the size of the upper and lower part according to litterature
    size_upper_part = math.floor(math.log(m,2))
    size_lower_part = m - size_upper_part
    # Add each element in the stream to the data structure
    for e in stream:
        eIpOrDomain = splitElement(e)
        eToHash = eIpOrDomain[0]
        if(domain):
            eToHash = eIpOrDomain[1]
        hashE = hashElement(eToHash, seed)
        upperpart = hashE >> size_lower_part
        lowerpart = hashE & ((2 ** size_lower_part) -1)
        # p is the position of the left most bit
        # We subtract it for 64 to get the position counting from left to right
        p = 64 - int(math.log(lowerpart,2))
        # j is the integer representation of the upperpart, we do not add one as python has 0 indexed 
        j = upperpart 
        # Use upper part as index in M and set it to the max of M[j] and p
        M[j] = max(M[j],p)
    return M

def getHarmonicMean(dataStucture):
    return s.harmonic_mean(dataStucture)

# takes harmonic mean hm and size of array, m as input, outputs the estimate
def getEstimate(hm, M, m):
    #alpha is the constant adjust to correct for bias. It's value is defined on Wiki: 
    #https://en.wikipedia.org/wiki/HyperLogLog#Operations
    alpha = 0.709
    E = alpha * (m**2) * (sum([2**-number for number in M])**-1)
    V = 0
    print(m)
    for i in range(len(M)): 
        if(M[i]==0):
            V += 1
    print(V)
    #Do the correct estimations
    EStar = E - 1.04*E/math.sqrt(m)

    return EStar

def splitElement(e):
    split = e.split()
    ip = split[0]
    domain = split[1]
    return ip, domain

web_traffic_stream = stream(STREAM_SIZE)
ds = buildHyperLogLog(web_traffic_stream, STREAM_SIZE, True, False,m)
print(ds)
hm = getHarmonicMean(ds)
print("Harmonic mean: " + str(hm))
E = getEstimate(hm, ds, m)
print("Estimate: " + str(E))


[11, 11, 12, 10, 10, 11, 11, 10, 13, 9, 10, 10, 10, 11, 12, 9, 12, 11, 9, 11, 11, 9, 10, 16, 10, 10, 13, 12, 13, 11, 14, 9, 10, 10, 10, 11, 13, 10, 10, 12, 14, 13, 13, 10, 12, 10, 13, 13, 12, 11, 11, 11, 11, 10, 10, 12, 12, 11, 13, 9, 11, 10, 12, 16]
Harmonic mean: 11.006567277006798
64
0
Estimate: 60474.449351526666


In [4]:
import numpy as np

## Count min sketch

d = 64
w = 64
def createM(d, w): 
    table = np.zeros([d, w])
    return table

def generateSeeds(d, w):
    seeds = np.random.randint(w, size = d)
    return seeds


def increment(seeds, key, table, d, w):
    for i in range(0, d):
        index = mmh3.hash(key, seeds[i]) % w
        table[i, index] = table[i, index]+1 
    return table

M = createM(d, w)
seeds = generateSeeds(d,w)
web_traffic_stream = stream(STREAM_SIZE)
for e in web_traffic_stream:
    increment(seeds,e,M,d,w)
    
def estimateResult(seeds, table, key, d, w):
        min_est = 100000
        for i in range(0, d):
            index = mmh3.hash(key, seeds[i]) % w
            #print("index" + str(index))
            #print("value" + str(table[i, index]))
            if table[i, index] < min_est:
                min_est = table[i, index]
        return min_est

def init_countMinSketchIp(d, w, seed):
    web_traffic_stream = stream(STREAM_SIZE)
    M = createM(d, w)
    
    for e in web_traffic_stream:
        split = e.split()
        ip = split[0]
        domain = split[1]
        M = increment(seed,ip,M,d,w)
    return M

def init_countMinSketchDomain(d, w, seed):
    web_traffic_stream = stream(STREAM_SIZE)
    M = createM(d, w)

    for e in web_traffic_stream:
        split = e.split()
        ip = split[0]
        domain = split[1]
        increment(seed,domain,M,d,w)
    return M

def countIp_countMinSketch(seed, M, targetIp, d, w):
    return (estimateResult(seed, M, targetIp, d,w))

def countDomain_countMinSketch(seed, M, targetDomain, d, w):
    return (estimateResult(seed, M, targetDomain, d,w))

def countAllDomain(seed, M, targetDomain, d, w):
    web_traffic_stream = stream(STREAM_SIZE)
    for e in web_traffic_stream:
        print(estimateResult(seed, M, targetDomain, d,w))
        



## Analysis

In [5]:
seed = generateSeeds(d,w)
M_ip = init_countMinSketchIp(d,w, seed)
M_domain = init_countMinSketchDomain(d,w, seed)


web_traffic_stream = stream(STREAM_SIZE)
e = next(web_traffic_stream).split() 
targetIp = e[0]
targetDomain = e[1]

print("Count ip: " + str(countIp_countMinSketch(seed, M_ip, targetIp, d, w)))
print("Count domain: " + str(countDomain_countMinSketch(seed, M_domain, targetDomain, d, w)))

e2 = next(web_traffic_stream).split() 
targetIp2 = e2[0]
targetDomain2 = e2[1]

print("Count ip2: " + str(countIp_countMinSketch(seed, M_ip, targetIp2, d, w)))
print("Count domain2: " + str(countDomain_countMinSketch(seed, M_domain, targetDomain2, d, w)))



Count ip: 8.0
Count domain: 281.0
Count ip2: 9.0
Count domain2: 510.0


In [6]:
seed = generateSeeds(d,w)
M = init_countMinSketchDomain(d, w, seed)

web_traffic_stream = stream(STREAM_SIZE)

targetDomain3 = 'python.org'
print("Count " + targetDomain3 +": " + str(countDomain_countMinSketch(seed, M, targetDomain3, d, w)))



Count python.org: 281.0


In [7]:
import mmh3
import math


class HyperLogLog: 
    def __init__(self):
        self._part = 10
        self._m = 2**self._part
        self._alpha = 0.7213/(1+1.079/self._m)
        self._M = [0 for i in range(0,self._m)]

    def add(self, elem):
        x = mmh3.hash(str(elem), signed=False)
        binary = '{0:032b}'.format(x)
        #part = int(math.log(64, 2))
        upperpart = binary[:self._part]
        lowerpart = binary[self._part:]
        j = int(upperpart,2)

        self._M[j] = max(self._M[j], len(lowerpart) - (int(lowerpart,2).bit_length()+1))

        pass
    
    def hashElement(element):
        # mmh3 64 outputs a 128 bit hash split in to two 64 bit outputs. 
        #For this purpose returning one of them will be sufficient
        a,b = mmh3.hash64(str(element), signed=False)
        return a

    def estimate(self):
        E = self._alpha*self._m**2*(sum([2**-number for number in self._M])**-1)
        V = self._M.count(0)
        
        if E <= (5/2)*self._m:
            if V != 0:
                EStar = self.LinearCounting(V)*2
            else:
                EStar = E
        elif E <= (1/30)*pow(2,32):
            EStar = E
        else: 
            EStar = -pow(2^32)*math.log2(1-E/pow(2,32))
        
        return EStar
    
    def LinearCounting(self, V):
        return self._m*math.log2(self._m/V)
    
    def Merge(self, other):
        res = HyperLogLog()
        res._M = [max(self._M[i],other._M[i]) for i in range(0,self._m)]
        return res

In [18]:
HLLDomain = HyperLogLog()
HLLIp = HyperLogLog()
web_traffic_stream = stream(STREAM_SIZE)

for e in web_traffic_stream:
    ip, domain = splitElement(e)
    HLLDomain.add(domain)
    HLLIp.add(ip)


In [19]:
HLL.estimate()

729.3825233413639

In [21]:
HLLIp.estimate()

678.1235522227261

In [22]:
HLLDomain.estimate()

5.776423039751338